<a href="https://colab.research.google.com/github/JuniorHZ19/Escalado/blob/main/EscalarVideosImanges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # **1) Descargar Librerias**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
from google.colab import drive, files
import os, shutil, subprocess
drive_mounted = False
temp_folder = 'tmp_frames'
result_folder = 'results'
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop

%cd ..

In [ ]:
#@markdown # **2) Descargar Modelos**
!pip install gdown

!gdown --id 1a6EZMTmsabf7C8YCTDqraoxNOSPsuXsD
!7z e ModelosESRGAN.zip

# Ruta al archivo ZIP que deseas descomprimir
archivo_zip = '/content/modelScaled.zip'

import zipfile
# Directorio de destino para la descompresión
directorio_destino = '/content/Real-ESRGAN/experiments/pretrained_models'  # Cambia '/content/destino/' al directorio donde deseas descomprimir los archivos


with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Descomprime los archivos en el directorio de destino
    zip_ref.extractall(directorio_destino)

# Verifica los archivos descomprimidos
import os
os.listdir(directorio_destino)


In [ ]:
#@markdown # **3) ** Montar Google Drive

if os.path.exists('/content/drive') is not True:
  drive.mount('/content/drive')

In [ ]:
#@markdown # **3.5) ** Montar el archivo de Google Drive
#@markdown Pon la ruta del archivo en tu google drive "content/drive/MyDrive/... y da click"



drive_mounted = True
drive_path = '/content/drive/MyDrive/data_dst_muestra.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive{drive_path}'
print("video path set:\n",input_path)

In [ ]:
#@markdown # **4) Extracter Frames del Video**

if os.path.isdir(temp_folder):
  shutil.rmtree(temp_folder)

os.mkdir(temp_folder)
print(f'Extracting Frames to: {temp_folder}')
cmd = [
       'ffmpeg',
       '-i',
       drive_path,
       '-qscale:v',
       '1',
       '-qmin',
       '1',
       '-qmax',
       '1',
       '-vsync',
       '0',
       f'{temp_folder}/frame_%08d.png'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    frame_count = len(os.listdir(f'/content/{temp_folder}'))
    print(f"Done, Extracted {frame_count} Frames")


In [ ]:
#@markdown # **4) Run ERSGAN y escalar**

#@markdown Camibar el nombre del modelo que queramos usar :

#@markdown RealESRGAN_x2plus,RealESRGAN_x4plus ,RealESRGAN_x4plus_anime_6B

%cd /content/Real-ESRGAN
frame_count = len(os.listdir(f'/content/{temp_folder}'))
print(f"Enhancing {frame_count} Frames with ESRGAN...")
cmd = [
       'python',
       'inference_realesrgan.py',
       '-n',
       'RealESRGAN_x4plus_anime_6B',
       '-i',
       f'../{temp_folder}',
       '--outscale',
       '2',

]
#process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# stdout, stderr = process.communicate()
# if process.returncode != 0:
#     print(stderr)
#     raise RuntimeError(stderr)
# else:
#     print("Done Extracting Frames")
!{' '.join(cmd)}
print("Done Enhancing Frames")
%cd ..

In [ ]:
#@markdown # **5) Crear el nuevo video de  los frames escalados
import subprocess
frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
  shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 30 #@param {type: "number"}
print(f"Recompiling {frame_count} Frames into Video...")
cmd = [
      'ffmpeg',
    '-framerate',str(fps),
     '-i',
       f'/content/Real-ESRGAN/results/frame_%08d_out.png',
      '-i',
      drive_path,

       '-map',
       '0:v',
       '-map',
       '1:a',
       '-c:v',
       'h264_nvenc',
       '-c:a',
        'mp3',

       '-b:v',
       '40M', #birate 5M (720P), 6M (720P:60FPS),8M(1080P), 10M (1080P),15M(1080P:60FPS) ,25M(2K),30M(2K:60FP) 40M(4K:60FP)


       f'{result_folder}/Mejorado{file_name}.mp4'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video")

In [ ]:
#@markdown # **6)Descargar archivo mejorado**
#@markdown Se pasara el archivo a nuestro drive

from google.colab import files

filename= os.listdir(result_folder)
if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/ESRGAN-Videos') is not True:
    os.makedirs('/content/drive/MyDrive/ESRGAN-Videos')
  shutil.move(f'{result_folder}/{filename[0]}',f'/content/drive/MyDrive/ESRGAN-Videos/{filename[0]}')
  print(f"MOviendo {filename[0]} el archivo a la carpeta ESRGAN-Videos en tu Drive")
else:
  files.download(f'{result_folder}/{filename[0]}')

In [ ]:
#@markdown # **7)Opcional Borrar archivos temporales  solo dar si vamos a escalar otro video
#@markdown  Solo dar click si vamos a escalar otro video

import os

# Ruta de la carpeta que contiene los archivos que deseas eliminar
carpetatemp = '/content/tmp_frames'
carpetaresult='/content/Real-ESRGAN/results'

# Lista todos los archivos en la carpeta
archivos = os.listdir(carpetatemp)

# Itera a través de los archivos y elimínalos
for archivo in archivos:
    ruta_completa = os.path.join(carpetatemp, archivo)
    if os.path.isfile(ruta_completa):
        os.remove(ruta_completa)
        print(f"Archivo eliminado: {ruta_completa}")

archivos = os.listdir(carpetaresult)
for archivo in archivos:
    ruta_completa = os.path.join(carpetaresult,archivo)
    if os.path.isfile(ruta_completa):
        os.remove(ruta_completa)
        print(f"Archivo eliminado: {ruta_completa}")
